# Azure OpenAI Local Tracing with SQLite

## Introduction

This notebook demonstrates **local tracing for Azure OpenAI API calls** using OpenTelemetry and SQLite storage. Unlike cloud-based tracing with Application Insights, this approach stores traces locally for offline development, testing, and debugging.

### What is Local Tracing?

Local tracing captures detailed telemetry about Azure OpenAI API interactions and stores them in a local SQLite database. This enables:

- **Offline Development**: No Azure dependencies for tracing
- **Cost-Effective Testing**: No Application Insights charges
- **Privacy**: All data stays on your machine
- **Fast Queries**: Direct SQL access to trace data
- **Debugging**: Detailed inspection of API calls and responses

### Key Features

**Custom Tracing Utilities:**
- Wrapper functions for synchronous and asynchronous API calls
- Built-in support for streaming responses
- Automatic span creation with semantic conventions
- SQLite storage with structured schema

**Captured Telemetry:**
- Request parameters (model, temperature, max_tokens, etc.)
- Full prompts and completions (configurable)
- Token usage (prompt, completion, total)
- Latency metrics (milliseconds)
- Error tracking and exceptions
- Streaming chunk counts

**SQLite Database:**
- Two tables: `spans` and `span_attributes`
- Foreign key relationships for data integrity
- Queryable with standard SQL
- Pandas integration for analysis

**Flexible Configuration:**
- Console output (optional)
- Batch vs immediate span processing
- Custom database paths
- Verbose logging control

### Tracing Workflow

```
1. Setup Tracing (SQLite + OpenTelemetry)
   ↓
2. Initialize Azure OpenAI Clients
   ↓
3. Make API Calls with Traced Wrappers
   ↓
4. Traces Stored in Local SQLite DB
   ↓
5. Query with SQL/Pandas
   ↓
6. Analyze Performance & Debug Issues
```

### What Gets Traced

**Request Details:**
- Model/deployment name
- Temperature, max_tokens, top_p
- Messages (system, user, assistant roles)
- Streaming vs non-streaming mode

**Response Details:**
- Full completion content
- Token usage (prompt, completion, total)
- Finish reason (stop, length, etc.)
- Choice count and index

**Performance Metrics:**
- End-to-end latency (ms)
- Chunk count (for streaming)
- Time to first token (streaming)

**Context:**
- Span IDs (OpenTelemetry format)
- Parent-child relationships
- Timestamps (ISO 8601)
- Status codes (OK, ERROR)

### Benefits

✅ **No Cloud Dependencies**: Works offline without Azure Monitor  
✅ **Cost-Free**: No Application Insights charges  
✅ **Fast Queries**: Direct SQL access with Pandas  
✅ **Privacy**: Data never leaves your machine  
✅ **Detailed Debugging**: Full prompt/response capture  
✅ **Performance Analysis**: Track latency trends over time  

### Use Cases

- **Local Development**: Test and debug without cloud costs
- **Performance Optimization**: Identify slow API calls
- **Token Usage Tracking**: Monitor and optimize token consumption
- **API Testing**: Compare different prompts and parameters
- **Debugging**: Reproduce issues with full conversation context
- **Cost Estimation**: Calculate API costs before production deployment

### Prerequisites

- Azure OpenAI endpoint and API key
- Python 3.8+
- OpenTelemetry packages (installed automatically)
- SQLite (built into Python)
- Environment variables configured (see Part 1.2)

## Table of Contents

1. [Part 1: Environment Setup](#part-1-environment-setup)
   - 1.1: Install Dependencies
   - 1.2: Configure Environment & Imports
   - 1.3: Initialize Tracing
   - 1.4: Initialize Azure OpenAI Clients
2. [Part 2: Synchronous API Calls](#part-2-synchronous-api-calls)
3. [Part 3: Asynchronous API Calls](#part-3-asynchronous-api-calls)
4. [Part 4: Streaming Responses](#part-4-streaming-responses)
5. [Part 5: Async Streaming](#part-5-async-streaming)
6. [Part 6: Query and Analyze Traces](#part-6-query-and-analyze-traces)
7. [Part 7: Database Management](#part-7-database-management)
8. [Summary and Best Practices](#summary-and-best-practices)

---

## Part 1: Environment Setup

### 1.1: Install Dependencies

Install required packages for local tracing with Azure OpenAI.

In [ ]:
%pip install -qU openai python-dotenv  opentelemetry-sdk opentelemetry-api

### 1.2: Configure Environment & Imports

Load environment variables and import tracing utilities.

**Required Environment Variables:**
- `AZURE_OPENAI_ENDPOINT_GPT_4o`: Your Azure OpenAI endpoint
- `AZURE_OPENAI_API_KEY_GPT_4o`: Your Azure OpenAI API key

**Tracing Utilities:**
- `setup_tracing()`: Initialize OpenTelemetry with SQLite exporter
- `traced_chat_completion()`: Sync API call wrapper
- `traced_chat_completion_async()`: Async API call wrapper
- `traced_chat_completion_streaming()`: Sync streaming wrapper
- `traced_chat_completion_streaming_async()`: Async streaming wrapper
- `flush_traces()`: Force export pending spans

In [ ]:
import os
import logging
from dotenv import load_dotenv
from openai import AzureOpenAI, AsyncAzureOpenAI

# Suppress verbose logging
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("opentelemetry").setLevel(logging.WARNING)

# Import tracing utilities
from tracing_utils import (
    setup_tracing,
    traced_chat_completion,
    traced_chat_completion_async,
    traced_chat_completion_streaming,
    traced_chat_completion_streaming_async,
    flush_traces
)

# Load environment variables
load_dotenv("../.env")

# Verify environment variables
required_vars = ["AZURE_OPENAI_ENDPOINT_GPT_4o", "AZURE_OPENAI_API_KEY_GPT_4o"]
missing = [var for var in required_vars if not os.getenv(var)]
if missing:
    print(f"⚠️  Warning: Missing environment variables: {missing}")
else:
    print("✅ Environment variables loaded successfully")
    print("✅ Imports successful")

### 1.3: Initialize Tracing

Setup tracing with SQLite storage and optional console output.

**Configuration Options:**
- `service_name`: Identifies your application in traces
- `enable_console`: Print traces to console (default: False)
- `enable_sqlite`: Store traces in SQLite database (default: True)
- `sqlite_db_path`: Path to SQLite database file
- `use_batch_processor`: Batch spans for better performance (default: True)
- `verbose`: Print configuration details

In [ ]:
# Initialize tracing
tracer = setup_tracing(
    service_name="my-azure-openai-app",
    enable_console=False,  # Set to True to see traces in console
    enable_sqlite=True,
    sqlite_db_path="my_traces.db",
    use_batch_processor=True,  # Set to False for immediate writes
    verbose=True
)

print("✅ Tracing initialized successfully")
print("📁 Traces will be stored in: my_traces.db")

### 1.4: Initialize Azure OpenAI Clients

Create synchronous and asynchronous clients for Azure OpenAI API calls.

In [ ]:
# Synchronous client
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT_GPT_4o"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY_GPT_4o"),
    api_version="2024-02-01"
)

# Asynchronous client
async_client = AsyncAzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT_GPT_4o"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY_GPT_4o"),
    api_version="2024-02-01"
)

deployment_name = "gpt-4o-lei"

print("✅ Azure OpenAI clients initialized")
print(f"   Deployment: {deployment_name}")
print(f"   Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT_GPT_4o')}")

---

## Part 2: Synchronous API Calls

Use `traced_chat_completion()` to make synchronous API calls with automatic tracing.

**What Gets Traced:**
- Request parameters (model, temperature, max_tokens)
- Full prompt messages (system, user roles)
- Completion content
- Token usage (prompt, completion, total)
- Latency (milliseconds)
- Status (success/failure)

**Return Values:**
- `response`: Standard OpenAI ChatCompletion object
- `latency`: End-to-end latency in seconds

In [ ]:
# Define messages
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What are the benefits of using Azure OpenAI?"}
]

# Make traced API call
response, latency = traced_chat_completion(
    client=client,
    tracer=tracer,
    messages=messages,
    model=deployment_name,
    temperature=0.7,
    max_tokens=200
)

print(f"📊 Performance Metrics:")
print(f"   Latency: {latency*1000:.2f}ms")
print(f"   Tokens: {response.usage.total_tokens} (prompt: {response.usage.prompt_tokens}, completion: {response.usage.completion_tokens})")
print(f"   Finish reason: {response.choices[0].finish_reason}")
print(f"\n💬 Response:\n{response.choices[0].message.content}")

print("\n✅ Trace stored in SQLite database")

---

## Part 3: Asynchronous API Calls

Use `traced_chat_completion_async()` for async/await pattern with automatic tracing.

**Benefits of Async:**
- Non-blocking I/O for concurrent operations
- Better throughput for multiple API calls
- Ideal for web servers and high-concurrency applications

**Usage:**
- Requires `await` keyword
- Returns same values as sync version
- All tracing features work identically

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Explain distributed tracing in 2 sentences."}
]

# Make async traced API call
response, latency = await traced_chat_completion_async(
    client=async_client,
    tracer=tracer,
    messages=messages,
    model=deployment_name,
    temperature=0.7,
    max_tokens=100
)

print(f"📊 Performance Metrics:")
print(f"   Latency: {latency*1000:.2f}ms")
print(f"   Tokens: {response.usage.total_tokens}")
print(f"\n💬 Response:\n{response.choices[0].message.content}")

print("\n✅ Async trace stored in SQLite database")

---

## Part 4: Streaming Responses

Use `traced_chat_completion_streaming()` for streaming responses with automatic tracing.

### Option 1: Collect All Chunks (Recommended for Tracing)

The traced wrapper collects all chunks, assembles the complete response, and then returns it. This ensures accurate token counting and complete content capture for the trace.

**What Gets Traced:**
- Complete assembled content
- Total chunk count
- End-to-end latency (including all chunks)
- All standard request/response attributes

**Return Values:**
- `content`: Complete assembled response text
- `chunks`: Total number of chunks received
- `latency`: Time from start to last chunk (seconds)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What is observability?"}
]

print("💬 Streaming Response:")
print("-" * 80)

# Use the traced function (collects all chunks, then prints)
content, chunks, latency = traced_chat_completion_streaming(
    client=client,
    tracer=tracer,
    messages=messages,
    model=deployment_name,
    temperature=0.7,
    max_tokens=500
)

print(content)
print("-" * 80)

print(f"\n📊 Streaming Metrics:")
print(f"   Latency: {latency*1000:.2f}ms")
print(f"   Chunks received: {chunks}")
print(f"   Content length: {len(content)} chars")
print(f"   Avg chunk size: {len(content)/chunks:.1f} chars/chunk")

print("\n✅ Streaming trace stored in SQLite database")

### Option 2: Real-Time Streaming Output

For **real-time streaming** (printing each chunk as it arrives), manually iterate over the stream and create a custom span.

**Trade-offs:**
- ✅ Immediate user feedback (better UX)
- ⚠️ Manual span management required
- ⚠️ Token counts not available until complete
- ⚠️ More complex error handling

In [ ]:
import time
from opentelemetry.trace import SpanKind, Status, StatusCode

messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What is observability? Explain in 3-4 sentences."}
]

print("💬 Real-Time Streaming Response:")
print("-" * 80)

# Create a traced span manually
with tracer.start_as_current_span(
    "azure_openai_streaming_realtime",
    kind=SpanKind.CLIENT,
    attributes={
        "gen_ai.system": "azure.openai",
        "gen_ai.request.model": deployment_name,
        "gen_ai.operation.name": "chat.completions.streaming",
        "gen_ai.request.streaming": True,
        "gen_ai.request.temperature": 0.7,
        "gen_ai.request.max_tokens": 500,
    }
) as span:
    start_time = time.time()

    try:
        # Add message attributes
        for idx, msg in enumerate(messages):
            span.set_attribute(f"gen_ai.prompt.{idx}.role", msg["role"])
            span.set_attribute(f"gen_ai.prompt.{idx}.content", msg["content"])

        # Make streaming API call
        stream = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            temperature=0.7,
            max_tokens=500,
            stream=True
        )

        # Stream and print in real-time
        full_content = ""
        chunk_count = 0

        for chunk in stream:
            chunk_count += 1
            if chunk.choices and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                full_content += content
                # Print each chunk immediately (real-time streaming!)
                print(content, end='', flush=True)

        print()  # New line after streaming completes
        print("-" * 80)

        # Calculate metrics
        latency = time.time() - start_time

        # Add final attributes to span
        span.set_attribute("gen_ai.completion.0.content", full_content)
        span.set_attribute("gen_ai.response.chunk_count", chunk_count)
        span.set_attribute("gen_ai.response.latency_ms", latency * 1000)
        span.set_status(Status(StatusCode.OK))

        print(f"\n📊 Real-Time Streaming Metrics:")
        print(f"   Latency: {latency*1000:.2f}ms")
        print(f"   Chunks received: {chunk_count}")
        print(f"   Total length: {len(full_content)} chars")

        print("\n✅ Real-time streaming trace stored in SQLite database")

    except Exception as e:
        span.set_status(Status(StatusCode.ERROR, str(e)))
        span.record_exception(e)
        raise

---

## Part 5: Async Streaming

Combine asynchronous execution with streaming responses for maximum performance.

### Option 1: Async Wrapper (Recommended)

Use `traced_chat_completion_streaming_async()` for automatic tracing with async/await pattern.

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What is telemetry data?"}
]

# Use the traced async streaming function
content, chunks, latency = await traced_chat_completion_streaming_async(
    client=async_client,
    tracer=tracer,
    messages=messages,
    model=deployment_name,
    temperature=0.7,
    max_tokens=150
)

print(f"📊 Async Streaming Metrics:")
print(f"   Latency: {latency*1000:.2f}ms")
print(f"   Chunks: {chunks}")
print(f"   Content length: {len(content)} chars")
print(f"   Avg chunk size: {len(content)/chunks:.1f} chars/chunk")

print(f"\n💬 Response:\n{content}")

print("\n✅ Async streaming trace stored in SQLite database")

### Option 2: Async Real-Time Streaming

For real-time output with async client, manually manage the async stream and span.

In [ ]:
import time

messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What is distributed tracing? Explain in 3 sentences."}
]

print("💬 Real-Time Async Streaming Response:")
print("-" * 80)

# Create a traced span manually
with tracer.start_as_current_span(
    "azure_openai_async_streaming_realtime",
    kind=SpanKind.CLIENT,
    attributes={
        "gen_ai.system": "azure.openai",
        "gen_ai.request.model": deployment_name,
        "gen_ai.operation.name": "chat.completions.async_streaming",
        "gen_ai.request.streaming": True,
        "gen_ai.request.temperature": 0.7,
        "gen_ai.request.max_tokens": 500,
    }
) as span:
    start_time = time.time()
    
    try:
        # Add message attributes
        for idx, msg in enumerate(messages):
            span.set_attribute(f"gen_ai.prompt.{idx}.role", msg["role"])
            span.set_attribute(f"gen_ai.prompt.{idx}.content", msg["content"])
        
        # Make async streaming API call
        stream = await async_client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            temperature=0.7,
            max_tokens=500,
            stream=True
        )
        
        # Stream and print in real-time
        full_content = ""
        chunk_count = 0
        
        async for chunk in stream:
            chunk_count += 1
            if chunk.choices and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                full_content += content
                # Print each chunk immediately (real-time streaming!)
                print(content, end='', flush=True)
        
        print()  # New line after streaming completes
        print("-" * 80)
        
        # Calculate metrics
        latency = time.time() - start_time
        
        # Add final attributes to span
        span.set_attribute("gen_ai.completion.0.content", full_content)
        span.set_attribute("gen_ai.response.chunk_count", chunk_count)
        span.set_attribute("gen_ai.response.latency_ms", latency * 1000)
        span.set_status(Status(StatusCode.OK))
        
        print(f"\n📊 Metrics:")
        print(f"  - Latency: {latency*1000:.2f}ms")
        print(f"  - Chunks received: {chunk_count}")
        print(f"  - Total length: {len(full_content)} chars")
        
    except Exception as e:
        span.set_status(Status(StatusCode.ERROR, str(e)))
        span.record_exception(e)
        raise

---

## Part 6: Query and Analyze Traces

Query the SQLite database to view and analyze stored traces using SQL and Pandas.

### Flush Pending Traces

Force flush all pending traces to database (important with BatchSpanProcessor).

In [ ]:
# Flush all pending traces
flush_traces(timeout=10)
print("✅ All pending traces flushed to database")
print("📊 Ready for querying and analysis")

### Trace Summary

Query the database for an overview of all captured traces.

**Queries Include:**
- Total span count
- Latest 10 traces with timestamps
- Performance statistics by operation type
- Streaming vs non-streaming comparison

In [ ]:
import sqlite3
import pandas as pd

def query_traces(db_path="my_traces.db"):
    """Query and display trace data from SQLite"""
    
    conn = sqlite3.connect(db_path)
    
    print("="*80)
    print("📊 TRACE SUMMARY")
    print("="*80)
    
    # Get total count
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM spans")
    total_spans = cursor.fetchone()[0]
    print(f"\n📈 Total spans stored: {total_spans}")
    
    # Get latest spans
    print("\n🕐 Latest 10 Traces:")
    df_spans = pd.read_sql_query("""
        SELECT 
            id,
            name,
            ROUND(duration_ms, 2) as duration_ms,
            status_code,
            datetime(created_at) as created_at
        FROM spans 
        ORDER BY created_at DESC 
        LIMIT 10
    """, conn)
    print(df_spans.to_string(index=False))
    
    # Get performance stats
    print("\n⚡ Performance Statistics:")
    df_stats = pd.read_sql_query("""
        SELECT 
            name,
            COUNT(*) as count,
            ROUND(AVG(duration_ms), 2) as avg_duration_ms,
            ROUND(MIN(duration_ms), 2) as min_duration_ms,
            ROUND(MAX(duration_ms), 2) as max_duration_ms
        FROM spans 
        GROUP BY name
    """, conn)
    print(df_stats.to_string(index=False))
    
    # Get streaming vs non-streaming stats
    print("\n🌊 Streaming vs Non-Streaming:")
    df_streaming = pd.read_sql_query("""
        SELECT 
            CASE 
                WHEN sa.value = 'True' THEN 'Streaming'
                ELSE 'Non-Streaming'
            END as mode,
            COUNT(DISTINCT s.id) as count,
            ROUND(AVG(s.duration_ms), 2) as avg_duration_ms
        FROM spans s
        LEFT JOIN span_attributes sa ON s.id = sa.span_id 
            AND sa.key = 'gen_ai.request.streaming'
        GROUP BY mode
    """, conn)
    print(df_streaming.to_string(index=False))
    
    conn.close()
    print("\n" + "="*80)

# Query the database
query_traces()

### Latest Span Details

View complete details of the most recent trace, including all attributes and metadata.

**Includes:**
- Span metadata (ID, name, duration, status)
- All attributes (prompts, completions, tokens, etc.)
- Timestamps and parent relationships

In [ ]:
def view_latest_span(db_path="my_traces.db"):
    """View all details of the latest span"""
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    print("="*80)
    print("🔍 LATEST SPAN DETAILS")
    print("="*80)
    
    cursor.execute("SELECT id FROM spans ORDER BY created_at DESC LIMIT 1")
    latest_span_id = cursor.fetchone()
    
    if latest_span_id:
        span_id = latest_span_id[0]
        
        # Get span details
        df_span = pd.read_sql_query(f"""
            SELECT * FROM spans WHERE id = {span_id}
        """, conn)
        
        print("\n📋 Span Information:")
        for col in df_span.columns:
            value = df_span[col].values[0]
            print(f"  {col:20s}: {value}")
        
        # Get attributes
        df_attrs = pd.read_sql_query(f"""
            SELECT key, value FROM span_attributes 
            WHERE span_id = {span_id}
            ORDER BY key
        """, conn)
        
        print(f"\n📋 Attributes ({len(df_attrs)} total):")
        for _, row in df_attrs.iterrows():
            key = row['key']
            value = row['value']
            # Show full value without truncation
            print(f"  {key}")
            print(f"    → {value}")
    else:
        print("\n❌ No spans found in database")
    
    conn.close()
    print("\n" + "="*80)

# View latest span
view_latest_span()

---

## Part 7: Database Management

Understanding the SQLite schema, foreign key relationships, and safe data management practices.

### Database Schema

Explore the relationship between `spans` and `span_attributes` tables and foreign key behavior.

**Schema Overview:**
- `spans`: Main table with span metadata
- `span_attributes`: Key-value attributes for each span
- Foreign key: `span_attributes.span_id` → `spans.id`

In [ ]:
def demonstrate_foreign_key_behavior(db_path="my_traces.db"):
    """Demonstrate what happens with foreign keys when deleting spans"""
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    print("="*80)
    print("🔍 FOREIGN KEY RELATIONSHIP DEMO")
    print("="*80)
    
    # Show current foreign key setting
    cursor.execute("PRAGMA foreign_keys")
    fk_status = cursor.fetchone()[0]
    print(f"\n⚙️  Foreign Keys Enabled: {bool(fk_status)}")
    
    # Show the schema
    print("\n📋 Table Relationships:")
    print("""
    spans table:
    ┌────┬──────────────────────┬─────────────┐
    │ id │ span_id (hex)        │ name        │  ← PRIMARY KEY
    └────┴──────────────────────┴─────────────┘
             ↑
             │ FOREIGN KEY
             │
    span_attributes table:
    ┌────┬─────────┬──────────────┬─────────┐
    │ id │ span_id │ key          │ value   │  ← span_id references spans.id
    └────┴─────────┴──────────────┴─────────┘
    """)
    
    # Count current spans and attributes
    cursor.execute("SELECT COUNT(*) FROM spans")
    span_count = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM span_attributes")
    attr_count = cursor.fetchone()[0]
    
    print(f"\n📊 Current Data:")
    print(f"  - Total spans: {span_count}")
    print(f"  - Total attributes: {attr_count}")
    
    if span_count > 0:
        # Show a sample span with its attributes
        cursor.execute("SELECT id, span_id, name FROM spans LIMIT 1")
        sample_span = cursor.fetchone()
        span_db_id, span_hex_id, span_name = sample_span
        
        print(f"\n📌 Sample Span:")
        print(f"  - Database ID (spans.id): {span_db_id}")
        print(f"  - OpenTelemetry ID (spans.span_id): {span_hex_id}")
        print(f"  - Name: {span_name}")
        
        cursor.execute(f"SELECT COUNT(*) FROM span_attributes WHERE span_id = {span_db_id}")
        attr_count_for_span = cursor.fetchone()[0]
        print(f"  - Attributes: {attr_count_for_span}")
        
        print(f"\n⚠️  What happens if you delete spans.id={span_db_id}?")
        print(f"""
        Current behavior (NO CASCADE):
        ❌ SQLite will REJECT the delete operation
        ❌ Error: "FOREIGN KEY constraint failed"
        ✅ Data integrity is protected
        ✅ No orphaned attributes
        
        To delete a span, you must:
        1. First delete all its attributes manually, OR
        2. Enable CASCADE DELETE in the schema
        """)
    
    conn.close()
    print("\n" + "="*80)

# Run the demo
demonstrate_foreign_key_behavior()

---

## Summary and Best Practices

### Key Takeaways

1. **Local Tracing**: SQLite-based tracing for offline development and testing
2. **Zero Cloud Costs**: No Application Insights charges during development
3. **Complete Privacy**: All trace data stays on your local machine
4. **Fast Queries**: Direct SQL access with Pandas integration
5. **Flexible Wrappers**: Support for sync, async, streaming, and async streaming

### What Gets Traced

**Request Details:**
- Model/deployment name
- Temperature, max_tokens, top_p
- Full message history (system, user, assistant)
- Streaming vs non-streaming mode

**Response Details:**
- Complete content (prompts and completions)
- Token usage (prompt, completion, total)
- Finish reason (stop, length, content_filter)
- Choice count and index

**Performance Metrics:**
- End-to-end latency (milliseconds)
- Chunk count (streaming)
- Time to first token (streaming)
- Average chunk size

**Context & Metadata:**
- Span IDs (OpenTelemetry format)
- Parent-child relationships
- Timestamps (ISO 8601)
- Status codes (OK, ERROR)
- Exception details (if errors occur)

### Best Practices

#### 1. Tracing Configuration
- ✅ Use BatchSpanProcessor for better performance (default)
- ✅ Call `flush_traces()` before script/notebook ends
- ✅ Disable console output in production (`enable_console=False`)
- ✅ Use descriptive service names for identification
- ✅ Suppress verbose logging for cleaner output

#### 2. API Call Patterns
- ✅ Use traced wrappers for automatic instrumentation
- ✅ Prefer async for concurrent operations
- ✅ Use streaming for long responses (better UX)
- ✅ Collect chunks for accurate tracing (Option 1)
- ✅ Use real-time streaming only when needed (Option 2)

#### 3. Database Management
- ✅ Enable CASCADE DELETE for easier cleanup
- ✅ Always set `PRAGMA foreign_keys = ON` in SQLite
- ✅ Implement data retention policies (delete old traces)
- ✅ Back up database before major changes
- ✅ Monitor database size for long-running applications

#### 4. Performance Analysis
- ✅ Query by operation type to identify patterns
- ✅ Track latency percentiles (P50, P95, P99)
- ✅ Monitor token usage trends over time
- ✅ Compare streaming vs non-streaming performance
- ✅ Identify outliers and investigate anomalies

#### 5. Debugging Workflows
- ✅ Use span IDs to trace specific requests
- ✅ Review full prompt/completion content
- ✅ Check token usage for optimization opportunities
- ✅ Analyze error traces for exception patterns
- ✅ Compare before/after traces for A/B testing

#### 6. Privacy & Security
- ✅ Keep database files secure (contains prompts/responses)
- ✅ Add database to `.gitignore` (avoid committing traces)
- ✅ Consider encrypting database for sensitive data
- ✅ Implement data retention policies (auto-delete old data)
- ✅ Review compliance requirements (GDPR, HIPAA, etc.)

### Database Schema

**spans table:**
```sql
CREATE TABLE spans (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    span_id TEXT NOT NULL,           -- OpenTelemetry span ID (hex)
    trace_id TEXT NOT NULL,          -- OpenTelemetry trace ID (hex)
    parent_span_id TEXT,             -- Parent span ID (for hierarchy)
    name TEXT NOT NULL,              -- Operation name
    kind INTEGER NOT NULL,           -- Span kind (CLIENT, SERVER, etc.)
    start_time INTEGER NOT NULL,     -- Start timestamp (nanoseconds)
    end_time INTEGER NOT NULL,       -- End timestamp (nanoseconds)
    duration_ms REAL NOT NULL,       -- Duration in milliseconds
    status_code INTEGER NOT NULL,    -- Status (OK=0, ERROR=1)
    status_message TEXT,             -- Error message (if any)
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
```

**span_attributes table:**
```sql
CREATE TABLE span_attributes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    span_id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT,
    FOREIGN KEY (span_id) REFERENCES spans(id) ON DELETE CASCADE
)
```

### Common SQL Queries

**Find slow API calls:**
```sql
SELECT name, duration_ms, datetime(created_at) as time
FROM spans 
WHERE duration_ms > 2000  -- Slower than 2 seconds
ORDER BY duration_ms DESC;
```

**Token usage by day:**
```sql
SELECT 
    DATE(created_at) as date,
    SUM(CAST(value AS INTEGER)) as total_tokens
FROM spans s
JOIN span_attributes sa ON s.id = sa.span_id
WHERE sa.key = 'gen_ai.usage.total_tokens'
GROUP BY DATE(created_at)
ORDER BY date DESC;
```

**Error analysis:**
```sql
SELECT 
    name,
    status_message,
    datetime(created_at) as time
FROM spans
WHERE status_code = 2  -- ERROR status
ORDER BY created_at DESC;
```

**Average latency by operation:**
```sql
SELECT 
    name,
    COUNT(*) as count,
    ROUND(AVG(duration_ms), 2) as avg_ms,
    ROUND(MIN(duration_ms), 2) as min_ms,
    ROUND(MAX(duration_ms), 2) as max_ms
FROM spans
GROUP BY name
ORDER BY avg_ms DESC;
```

### Migration to Cloud Tracing

When ready for production, migrate to Azure Monitor (Application Insights):

1. **Keep Local Tracing for Development:**
   - Continue using SQLite for testing
   - Use for cost-free experimentation
   - Validate changes before deployment

2. **Add Azure Monitor for Production:**
   - See `01_azure_foundry_tracing.ipynb` for setup
   - Automatic export to Application Insights
   - Built-in dashboards and alerting
   - KQL queries for advanced analysis

3. **Hybrid Approach:**
   - Use both local and cloud tracing
   - Local for development/debugging
   - Cloud for production monitoring
   - Compare results between environments

### Troubleshooting

| Issue | Solution |
|-------|----------|
| No traces in database | Call `flush_traces()` before querying |
| Foreign key errors | Run `PRAGMA foreign_keys = ON` in SQLite |
| Large database size | Implement data retention (delete old traces) |
| Slow queries | Add indexes on frequently queried columns |
| Missing attributes | Check traced wrapper is used correctly |
| Incomplete content | Use Option 1 (collect chunks) for streaming |

### Next Steps

1. **Customize Tracing**: Modify `tracing_utils.py` for your needs
2. **Add Indexes**: Optimize database for your query patterns
3. **Implement Retention**: Auto-delete old traces (e.g., > 30 days)
4. **Build Dashboards**: Create Jupyter dashboards with plotly/matplotlib
5. **Export Data**: Export to CSV/JSON for external analysis
6. **Migrate to Cloud**: Set up Azure Monitor for production workloads

### Additional Resources

- [OpenTelemetry Python SDK](https://opentelemetry.io/docs/instrumentation/python/)
- [OpenTelemetry Semantic Conventions](https://opentelemetry.io/docs/specs/semconv/)
- [SQLite Documentation](https://www.sqlite.org/docs.html)
- [Pandas SQL Guide](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)
- [Azure Monitor OpenTelemetry](https://learn.microsoft.com/azure/azure-monitor/app/opentelemetry-enable)

### Related Notebooks

- `01_azure_foundry_tracing.ipynb`: Cloud-based tracing with Application Insights
- `../01_agent/`: Azure AI agent examples
- `../05_evaluation/`: Agent evaluation and metrics
- `../02_governance/`: Compliance and governance patterns